In [7]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [8]:
EQUITY_QUINTILES = [0, 0.3, 0.37, 0.44, 0.53, 1]

In [9]:
gdf_locations = gpd.read_file('../joined-data/simplified_matches_4326_full.geo.json').to_crs(3857)
gdf_ct = gpd.read_file('../ct-data/ct-data-all.geo.json').to_crs(3857)

Finding 3: Average number of agency spaces (ie. individual locations) within a census tract area (ie. completely within a census tract or no more than 800 meters away from it). We want to break this down by each of the quantiles of the equity index as well.

In [10]:
# Assign quintile labels based on equity index
gdf_ct['quintile'] = pd.cut(
    gdf_ct['Equity Index'], bins=EQUITY_QUINTILES, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'], include_lowest=True
)

result = []

# Compute averages for all quintiles combined and by region
all_quintiles = {
    'quintile': 'Q_all',
    'total_avg': gdf_ct['total_count_800m'].mean()
}
for region in ['Peel', 'Toronto', 'York']:
    all_quintiles[region] = gdf_ct.loc[gdf_ct['region'] == region, 'total_count_800m'].mean()
result.append(all_quintiles)

# Compute averages for each quintile and by region
for quintile in ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']:
    quintile_data = gdf_ct[gdf_ct['quintile'] == quintile]
    row = {
        'quintile': quintile,
        'total_avg': quintile_data['total_count_800m'].mean()
    }
    for region in ['Peel', 'Toronto', 'York']:
        row[region] = quintile_data.loc[quintile_data['region'] == region, 'total_count_800m'].mean()
    result.append(row)

# Save results to a DataFrame and CSV
average_locations_df = pd.DataFrame(result)
average_locations_df = average_locations_df.round(4)  # Round all numbers to 4 decimal places
average_locations_df.to_csv('../report-results/3_avg_locations.csv', index=False)

average_locations_df

,quintile,total_avg,Peel,Toronto,York
0,Q_all,11.3859,3.3085,17.3333,5.3182
1,Q1,11.8609,1.6042,18.3094,2.4651
2,Q2,11.0150,2.3492,20.7284,6.7143
3,Q3,10.7554,3.1882,20.5161,5.9455
4,Q4,8.5100,5.1148,11.0374,6.5312
5,Q5,14.7871,5.0000,17.1333,2.9167


Finding 4: Percentage of agency spaces near existing and future transit stations (ie. within 800m). We want this broken down by owned (CORE) and leased (CLRE)

Finding 5: Percentage of agency spaces which are rented (CLRE), broken down by municipality and equity index quintile